In [ ]:
require 'torch'
require 'gnuplot'
require 'rnn'
require 'image'
Plot = require 'itorch.Plot'

In [ ]:
gpu=1
if gpu>0 then
  print("CUDA ON")
  require 'cutorch'
  require 'cunn'
  cutorch.setDevice(gpu)
end

In [ ]:
nIters = 200
batchSize = 80
rho = 10
hiddenSize = 300
nIndex = 1
lr = 0.0001
nPredict=200

rnn = nn.Sequential()
   :add(nn.Linear(nIndex, hiddenSize))
   :add(nn.FastLSTM(hiddenSize, hiddenSize))
   :add(nn.NormStabilizer())
   :add(nn.Linear(hiddenSize, nIndex))
   :add(nn.HardTanh())
rnn = nn.Sequencer(rnn)
rnn:training()
---- print(rnn)

In [ ]:
if gpu>0 then
  rnn=rnn:cuda()
end

criterion = nn.MSECriterion()
if gpu>0 then
  criterion=criterion:cuda()
end

ii=torch.linspace(0,200, 2000)
sequence=torch.cos(ii)
if gpu>0 then
  sequence=sequence:cuda()
end

In [ ]:
offsets = {}
for i=1,batchSize do
   table.insert(offsets, math.ceil(math.random()* (sequence:size(1)-rho) ))
end
offsets = torch.LongTensor(offsets)
if gpu>0 then
  offsets=offsets:cuda()
end

--- local gradOutputsZeroed = {}
gradOutputsZeroed = {}
for step=1,rho do
  gradOutputsZeroed[step] = torch.zeros(batchSize,1)
  if gpu>0 then
    gradOutputsZeroed[step] = gradOutputsZeroed[step]:cuda()
  end
end

In [ ]:
local iteration = 5
while iteration < nIters do
   local inputs, targets = {}, {}
    inputs2, targets2 = {}, {}
   for step=1,rho do
      inputs[step] = sequence:index(1, offsets):view(batchSize,1)
      inputs2[step] = sequence:index(1, offsets):view(batchSize,1)
      offsets:add(1)
      for j=1,batchSize do
         if offsets[j] > sequence:size(1) then
            offsets[j] = 1
         end
      end
      targets2[step] = sequence:index(1, offsets)
      targets[step] = sequence:index(1, offsets)
   end
   rnn:zeroGradParameters()
   local outputs = rnn:forward(inputs)
   local err = criterion:forward(outputs[rho], targets[rho])
   ---print(string.format("Iteration %d ; NLL err = %f ", iteration, err))
   local gradOutputs = criterion:backward(outputs[rho], targets[rho])
   gradOutputsZeroed[rho] = gradOutputs
   local gradInputs = rnn:backward(inputs, gradOutputsZeroed)
   rnn:updateParameters(lr)
   iteration = iteration + 1
end

In [ ]:
rnn:evaluate()
predict=torch.FloatTensor(nPredict)
if gpu>0 then
  predict=predict:cuda()
end
for step=1,rho do
  predict[step]= sequence[step]
end

start = {}
iteration=0
while rho + iteration < nPredict do
  for step=1,rho do
    start[step] = predict:index(1,torch.LongTensor({step+iteration})):view(1,1)
  end

  output = rnn:forward(start)

  predict[iteration+rho+1] = (output[rho]:float())[1][1]

  iteration = iteration + 1
end

In [ ]:
-- scatter plots
plot = Plot():circle(torch.range(1, predict.size(predict)[1]), 
                predict, 'red', 'Loss Function'):draw()
plot:line(torch.range(1, predict.size(predict)[1]), 
                predict, 'red', ''):redraw()
plot:title('Scatter Plot Demo'):redraw()
plot:xaxis('Iterations'):yaxis('Negative Log-Likelihood'):redraw()
plot:legend(true)
plot:redraw()

# Try 2

In [10]:
require 'rnn'
require 'cunn'
cuda = true

print("Running batch version of an RNN with continuous outcome")

function build_data()
    local inputs = {}
    local targets = {}
    --Use previously created and saved data
    for i = 1, dsSize do
      -- populate both tables to get ready for training
      -- local input = torch.randn(batchSize, inputSize)
        local input = torch.Tensor({{1, 2, 3, 4}, {0, 1, 1, 3}})
        local target = torch.randn(batchSize)
        if cuda then
            input = input:float():cuda()
            target = target:float():cuda()
        end
            table.insert(inputs, input)
            table.insert(targets, target)
    end
    return inputs, targets
end

function build_network(inputSize, hiddenSize, outputSize)
    -- This works for the discrete
    rnn = nn.Sequential()
    rnn:add(nn.LookupTableMaskZero(4, hiddenSize))
    rnn:add(nn.SeqLSTM(hiddenSize, hiddenSize))
    rnn:add(nn.Select(1, -1))                       --- Select last embedding layer of sequence
    rnn:add(nn.Linear(hiddenSize, outputSize))
   -- wrap this in a Sequencer such that we can forward/backward 
   -- entire sequences of length seqLength at once
   rnn = nn.Sequencer(rnn)
   if cuda then
      rnn:cuda()
   end
   return rnn
end



inputSize = 6
-- Larger numbers here mean more complex problems can be solved, but can also over-fit. 256 works well for now
hiddenSize = 8
-- We want the network to classify the inputs using a one-hot representation of the outputs
outputSize = 2

-- the dataset size is the total number of examples we want to present to the LSTM 
dsSize=10

-- We present the dataset to the network in batches where batchSize << dsSize
batchSize=2

-- And seqLength is the length of each sequence, i.e. the number of "events" we want to pass to the LSTM
-- to make up a single example. I'd like this to be dynamic ideally for the YOOCHOOSE dataset..
seqLength=8

-- number of target classes or labels, needs to be the same as outputSize above
-- or we get the dreaded "ClassNLLCriterion.lua:46: Assertion `cur_target >= 0 && cur_target < n_classes' failed. "
nClass = 2

-- two tables to hold the *full* dataset input and target tensors

inputs, targets = build_data()
rnn = build_network(inputSize, hiddenSize, outputSize)

print('Example of inputs and outputs for a batch of data:')
print(inputs[1], targets[2])

crit = nn.MSECriterion()
--- crit = nn.ClassNLLCriterion()
seqC = nn.SequencerCriterion(crit)
if cuda then
   crit:cuda()
   seqC:cuda()
end

rnn:training()

Start training	


Epoch	0	loss	0.56662445068359 in 	0.20637607574463 s	


Epoch	10	loss	0.54920355677605 in 	0.031195163726807 s	


Epoch	20	loss	0.54874401688576 in 	0.033711910247803 s	


Epoch	30	loss	0.54840999245644 in 	0.034494876861572 s	


...

Epoch	60	loss	0.54730515480042 in 	0.033943891525269 s	


Epoch	70	loss	0.54683074355125 in 	0.033765077590942 s	


Epoch	80	loss	0.54625191092491 in 	0.036449909210205 s	


Epoch	90	loss	0.54551874995232 in 	0.030251979827881 s	


Epoch	100	loss	0.54455727934837 in 	0.0414879322052 s	


In [ ]:
-- scatter plots
plot = Plot():circle(torch.range(1, #loss), 
                loss, 'red', 'Loss Function'):draw()
plot:line(torch.range(1, #loss), 
                loss, 'red', ''):redraw()
plot:title('Scatter Plot Demo'):redraw()
plot:xaxis('Iterations'):yaxis('Negative Log-Likelihood'):redraw()
plot:legend(true)
plot:redraw()